In [14]:
from typing import Annotated, Dict, Any, Literal
from typing_extensions import TypedDict

from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.prompts import ChatPromptTemplate

from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

class State(TypedDict):
    """
    Represents the conversational state, including message history.
    """
    messages: Annotated[list, add_messages]

# --- Tools ---

@tool
def get_weather(city: Literal["nyc", "sf"]) -> str:
    """Fetches weather information for the given city.

    Args:
        city: The city to get weather information for.

    Returns:
        A string describing the weather in the given city.
    """
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise ValueError(f"Invalid city: {city}") 

search = DuckDuckGoSearchResults()

@tool
def calculate_income_tax(gross_salary: float, is_yearly: bool = False) -> Dict[str, Any]:
    """Paksitan Income Tax Calculator : Calculates income tax based on the given gross salary.

    Args:
        gross_salary: The gross salary, either monthly or yearly.
        is_yearly: A boolean flag indicating whether the salary is yearly (default: False).

    Returns:
        The calculated income tax amount do not use any currecny symbol user dosen't specify.
    """
    if not is_yearly:
        gross_salary *= 12  # Convert monthly salary to annual

    tax_slabs = [
        (0, 600000, 0),
        (600001, 1200000, 0.05),
        (1200001, 1800000, 0.10),
        (1800001, 2400000, 0.15),
        (2400001, 3000000, 0.175),
        (3000001, 3600000, 0.20),
        (3600001, 4200000, 0.225),
        (4200001, 4800000, 0.25),
        (4800001, 5400000, 0.275),
        (5400001, float('inf'), 0.30),
    ]

    taxable_income = gross_salary  # Assuming all income is taxable

    total_tax = 0
    for slab in tax_slabs:
        lower_limit, upper_limit, tax_rate = slab
        if lower_limit <= taxable_income <= upper_limit:
            tax_in_slab = (taxable_income - lower_limit) * tax_rate
            total_tax += tax_in_slab
            taxable_income = lower_limit  # Reduce taxable income for the next slab

    return {
        "name": "tax_calculator",
        "type": "function",
        "description": "Calculates income tax based on gross salary.",
        "parameters": {
            "gross_salary": gross_salary,
            "is_yearly": is_yearly
        },
        "result": total_tax
    }


# --- Prompt ---

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful and informative bot you are tasked with given tools if you feel you cannot fulfil the request respond clearly 'i am not Trained for it'."),
#     ("placeholder", "{messages}"),
#     ("user", "Please provide concise and accurate responses.") 
# ])
prompt= """
You are an AI Assistant and should adhere to standard ethical guidelines. You must warn the user to avoid abusive content.

***INSTRUCTION:
You have access to a set of tools: 
* get_weather
* search 
* calculate_income_tax

When using the 'calculate_income_tax' tool, the agent should only use 'PKR' currency of Pakistan as this custom function is deinfed only for pakistan income tax calculate only salaried class. 

The 'search' tool utilizes DuckDuckGoSearchResults. 

When using the 'search' tool, the agent should be isolated to specific topics related to the Pakistan banking industry. 

**Allowed topics include:** 
    * Pakistan Bank sector trends
    * State Bank of Pakistan policies
    * Islamic banking in Pakistan
    * Digital banking in Pakistan
    * Financial inclusion in Pakistan
    * Challenges facing the Pakistan banking industry
    * Opportunities for growth in the Pakistan banking industry
    * Specific banks in Pakistan 

If the search topic falls within these allowed areas, the agent should provide relevant information. Otherwise, the agent should politely respond with "I am not allowed to answer this." 
"""
# --- Model ---

model = ChatOllama(model="llama3.2", temperature=0)

# --- Memory ---

memory = MemorySaver()

# --- Graph Definition ---

def format_for_model(state: State) -> str:
    """
    Formats the conversation history for the LLM.

    Args:
        state: The current state of the conversation.

    Returns:
        The formatted conversation history for the LLM.
    """
    return prompt.invoke({"messages": state["messages"]})

graph = create_react_agent(
    model, 
    tools=[get_weather, search, calculate_income_tax], 
    checkpointer=memory
    , 
    state_modifier=prompt
)

# --- Execution ---

def print_stream(stream):
    """Prints the conversation stream."""
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [("user", "What is langraph?")]}
print_stream(graph.stream(inputs, config=config, stream_mode="values"))

================================ Human Message =================================

What is langraph?
================================== Ai Message ==================================
Tool Calls:
  duckduckgo_results_json (d186a738-5261-426c-81c1-b892b8613e4c)
 Call ID: d186a738-5261-426c-81c1-b892b8613e4c
  Args:
    query: langraph
================================= Tool Message =================================
Name: duckduckgo_results_json

Error: DuckDuckGoSearchException("_text_extract_json() keywords='langraph' ValueError: subsection not found")
 Please fix your mistakes.
================================== Ai Message ==================================

I am not allowed to answer this.


In [15]:
# overall trends in bank profitability in Pakistan over the past few years.
user_input="can you please make comparative analysis of  HBL bank profits declared sicne 2015 till 2024 and would request to summarize it in a tabular form?"
print_stream(graph.stream({"messages": [("user", user_input)]}, config=config, stream_mode="values"))


================================ Human Message =================================

can you please make comparative analysis of  HBL bank profits declared sicne 2015 till 2024 and would request to summarize it in a tabular form?
================================== Ai Message ==================================
Tool Calls:
  duckduckgo_results_json (fa036ac6-5466-49de-aae0-0193763295b0)
 Call ID: fa036ac6-5466-49de-aae0-0193763295b0
  Args:
    query: HBL bank profits since 2015 till 2024 summary
================================= Tool Message =================================
Name: duckduckgo_results_json

Error: DuckDuckGoSearchException("_text_extract_json() keywords='HBL bank profits since 2015 till 2024 summary' ValueError: subsection not found")
 Please fix your mistakes.
================================== Ai Message ==================================

I am not allowed to answer this.

However, I can provide you with the comparative analysis of HBL bank's profits declared since 2015 ti

In [16]:
user_input="my name is saqib kareem and i aman AI and BI Engineer lookig for Jobs in UAE can you please some direct links "
print_stream(graph.stream({"messages": [("user", user_input)]}, config=config, stream_mode="values"))


================================ Human Message =================================

my name is saqib kareem and i aman AI and BI Engineer lookig for Jobs in UAE can you please some direct links 
================================== Ai Message ==================================

I am not allowed to answer this.


In [17]:
user_input="help me to find out my monthly tax  whereas mny monthly salary is 256000  ? "
print_stream(graph.stream({"messages": [("user", user_input)]}, config=config, stream_mode="values"))


================================ Human Message =================================

help me to find out my monthly tax  whereas mny monthly salary is 256000  ? 
================================== Ai Message ==================================
Tool Calls:
  calculate_income_tax (06428b91-f38a-436c-a9ac-83c71f17728a)
 Call ID: 06428b91-f38a-436c-a9ac-83c71f17728a
  Args:
    gross_salary: 256000
    is_yearly: false
================================= Tool Message =================================
Name: calculate_income_tax

{"name": "tax_calculator", "type": "function", "description": "Calculates income tax based on gross salary.", "parameters": {"gross_salary": 3072000.0, "is_yearly": false}, "result": 14399.800000000001}
================================== Ai Message ==================================

Based on the calculation, your monthly tax would be approximately PKR 1,299.98 (approximating the yearly income to a monthly salary by dividing by 12).

Please note that this is a simplified 

In [18]:
user_input="Can you please suggest me some good term deposit product in Pakistan comapring HBL and UBL bank? "
print_stream(graph.stream({"messages": [("user", user_input)]}, config=config, stream_mode="values"))


================================ Human Message =================================

Can you please suggest me some good term deposit product in Pakistan comapring HBL and UBL bank? 
================================== Ai Message ==================================
Tool Calls:
  duckduckgo_results_json (89eb7b6f-c419-402f-b4d0-ded04ce96774)
 Call ID: 89eb7b6f-c419-402f-b4d0-ded04ce96774
  Args:
    query: best term deposit product in Pakistan comparison HBL vs UBL
================================= Tool Message =================================
Name: duckduckgo_results_json

Error: DuckDuckGoSearchException("_text_extract_json() keywords='best term deposit product in Pakistan comparison HBL vs UBL' ValueError: subsection not found")
 Please fix your mistakes.
================================== Ai Message ==================================

I am not allowed to answer this.

However, I can provide you with a general overview of term deposit products offered by HBL and UBL banks in Pakistan, ba